In [1]:
import numpy as np
import h5py
import sys
sys.path.append('F:\Linux')
import illustris_python as il
from sklearn.neighbors import KDTree
import matplotlib.pyplot as plt

#match disk halo between TNG & illustris-1
tngdisk = np.load('f:/Linux/localRUN/diskID_4WP.npy')
il1disk = np.load('f:/Linux/localRUN/diskID_il1.npy')

tng_barred = np.load('f:/Linux/localRUN/barredID_4WP_TNG.npy')
il1_barred = np.load('f:/Linux/localRUN/barredID_il1.npy')

Groups_tng = il.func.loadSubhalos('TNG', 99, "SubhaloGrNr")
Groups_il1 = il.func.loadSubhalos('il1', 135, "SubhaloGrNr")

tng_diskGIDs = Groups_tng[tngdisk]
il1_diskGIDs = Groups_il1[il1disk]

tng_barredGID = Groups_tng[tng_barred]
il1_barredGID = Groups_il1[il1_barred]

#load TNG groups informations
cm_TNG = il.func.loadhalos('TNG', 99, 'GroupPos')
tng_subCM = il.func.loadSubhalos('TNG', 99, 'SubhaloCM')
mass_TNG = il.func.loadhalos('TNG', 99, 'GroupMassType')[:, 1] / 0.6774

#build illustris halo as a tree
cm_il1 = il.func.loadhalos('il1', 135, 'GroupPos')
il1_subCM = il.func.loadSubhalos('il1', 135, 'SubhaloCM')
mass_il1 = il.func.loadhalos('il1', 135, 'GroupMassType')[:, 1] / 0.704

mass_TNG = np.log10(mass_TNG * 10 ** 10)
mass_il1 = np.log10(mass_il1 * 10 ** 10)

#Is centre galaxy
tng_GFSub = il.func.loadhalos('TNG', 99, 'GroupFirstSub')
il1_GFSub = il.func.loadhalos('il1', 135, 'GroupFirstSub')
def isCentral(haloID, simu='tng'):
    if simu == 'tng':
        GID = Groups_tng[haloID]
        GFSub = tng_GFSub[GID]
    else:
        GID = Groups_il1[haloID]
        GFSub = il1_GFSub[GID]
        
    if GFSub == haloID:
        return True
    else:
        return False


In [2]:
tree = KDTree(cm_il1, leaf_size = 2)
sml_threshold = 2.5 * 1000
big_threhold = 12.5 * 1000

In [3]:
#tngID, il1ID, dis, absMass
sameHalo = [[], [], [], []]
for groupID in np.unique(tng_diskGIDs):
    sameHalo[0].append([groupID])
    dis, index = tree.query(cm_TNG[groupID].reshape(1, -1), k=10)

    il1_id = []
    distance = []
    absMass = []
    for i in range(10):
        if abs(mass_TNG[groupID] - mass_il1[index[0, i]]) <= mass_TNG[groupID]*0.5:
            il1_id.append(index[0, i])
            distance.append(dis[0, i])
            absMass.append(abs(mass_TNG[groupID] - mass_il1[index[0, i]]))
        else:
            il1_id.append(-1)
            distance.append(dis[0, i])
            absMass.append(abs(mass_TNG[groupID] - mass_il1[index[0, i]]))

    sameHalo[1].append(il1_id)
    sameHalo[2].append(distance)
    sameHalo[3].append(absMass)

sameHalo = np.array(sameHalo)

In [4]:
data = {}
for groupID in np.unique(tng_diskGIDs):
    dis, index = tree.query(cm_TNG[groupID].reshape(1, -1), k=10000)
    if mass_TNG[groupID] >= 12:
        index = index[dis < big_threhold]
    else:
        index = index[dis < sml_threshold]
    index = index[(mass_il1[index] > (mass_TNG[groupID]*0.8))]
    data[groupID] = index
    

In [5]:
data

{0: array([0], dtype=int64),
 1: array([1], dtype=int64),
 2: array([2], dtype=int64),
 3: array([3], dtype=int64),
 4: array([4], dtype=int64),
 5: array([9], dtype=int64),
 6: array([   5, 2771], dtype=int64),
 7: array([3086], dtype=int64),
 8: array([8], dtype=int64),
 9: array([6], dtype=int64),
 10: array([10], dtype=int64),
 11: array([11], dtype=int64),
 12: array([4041, 7568,   12], dtype=int64),
 13: array([  13, 8948, 5040], dtype=int64),
 14: array([  15, 5950, 5976, 5059, 7528], dtype=int64),
 15: array([  14, 4198, 7446], dtype=int64),
 16: array([  34, 6155, 8098, 3786], dtype=int64),
 17: array([  18, 6303, 6009], dtype=int64),
 18: array([  17, 5822, 8464, 3894], dtype=int64),
 19: array([  16, 2735], dtype=int64),
 20: array([   22, 10285,  2520,  7315,  4974,  7505, 10401,  1646,  9261,
         5523,  4173,  3329], dtype=int64),
 21: array([   19, 10735,  9750,  4854,   459,  1563,  9061], dtype=int64),
 22: array([   24,  8044,  7114,  3455,  8168,  4043, 11937,  1

In [6]:
len(data.keys())

1019

In [7]:
np.save('f:/Linux/localRUN/Match/TNG_match_dict.npy', data)

In [11]:
a=np.load('f:/Linux/localRUN/Match/TNG_match_dict.npy',allow_pickle=1).item()

In [13]:
a[0][0]

0